In [1]:
import pandas as pd
from fbprophet import Prophet

In [2]:

df = pd.read_csv('../data/lrpd-clean.csv')
df['INCIDENT_DATE'] = pd.to_datetime(df['INCIDENT_DATE'])
df2 = df.groupby(pd.Grouper(key='INCIDENT_DATE', freq='D')).size().reset_index(name='INCIDENT_COUNT')
df2.columns = ['ds', 'y']
df2.tail()

,ds,y
2237,2023-02-16,42
2238,2023-02-17,32
2239,2023-02-18,25
2240,2023-02-19,36
2241,2023-02-20,30


In [3]:
#Get all holidays in US as dataframe from 2017 to 2022
import holidays

us_holidays = holidays.UnitedStates(years=[2017, 2018, 2019, 2020, 2021, 2022, 2023])
us_holidays = pd.DataFrame.from_dict(us_holidays, orient='index').reset_index()
us_holidays.columns = ['ds', 'holiday']
us_holidays['ds'] = pd.to_datetime(us_holidays['ds'])


In [4]:
#let train be 80% of the data
train = df2[:int(0.8 * (len(df2)))]
test = df2[int(0.8 * (len(df2))):]
train.tail(), test.head()

(             ds   y
 1788 2021-11-24  33
 1789 2021-11-25  16
 1790 2021-11-26  48
 1791 2021-11-27  35
 1792 2021-11-28  25,
              ds   y
 1793 2021-11-29  47
 1794 2021-11-30  45
 1795 2021-12-01  45
 1796 2021-12-02  43
 1797 2021-12-03  45)

In [5]:

from statsmodels.tools.eval_measures import rmse
import itertools

param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative'],
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []

for params in all_params:
    m = Prophet(**params,
                holidays=us_holidays,
                yearly_seasonality=True,
                ).fit(train)
    future = m.make_future_dataframe(periods=test.shape[0])
    forecast = m.predict(future)
    predictions = forecast[int(0.8 * (len(forecast))):]['yhat']
    predictions = predictions[:len(test)]
    actual = test['y']
    rmses.append(rmse(actual, predictions))

tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
tuning_results

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,seasonality_mode,rmse
0,0.001,0.01,0.01,additive,10.255888
1,0.001,0.01,0.01,multiplicative,9.627794
2,0.001,0.01,0.10,additive,10.303324
3,0.001,0.01,0.10,multiplicative,10.237061
4,0.001,0.01,1.00,additive,10.280794
...,...,...,...,...,...
123,0.500,10.00,0.10,multiplicative,9.143570
124,0.500,10.00,1.00,additive,9.189433
125,0.500,10.00,1.00,multiplicative,9.157866
126,0.500,10.00,10.00,additive,9.074446
